# Обнаруэение человека с уличных камер  

Идентифицикация людей на близком расстоянии с помощью данных с уличных камер и OpenCV HOGDescriptor. Работа не стабильная

In [13]:
import cv2
import numpy as np
import IPython

In [14]:
def analyze(frame):
    # initialize the HOG descriptor/person detector
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    # using a greyscale picture, also for faster detection
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

    # detect people in the image
    # returns the bounding boxes for the detected objects
    boxes, weights = hog.detectMultiScale(gray, winStride=(8,8) )

    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])

    for (xA, yA, xB, yB) in boxes:
        # display the detected boxes in the colour picture
        cv2.rectangle(frame, (xA, yA), (xB, yB),
                          (0, 0, 255), 2)
    return frame

In [15]:
cap = cv2.VideoCapture("https://restreamer.vms.evo73.ru/918335436b92ac26/stream1.ts") # IP webcam
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()

    processedFrame=analyze(frame)

    # Display the resulting frame
    cv2.imshow('frame',processedFrame)
    if cv2.waitKey(1) & 0xFF == ord(' '):
        break

# When everything done, release the capture
cap.release()

cv2.destroyAllWindows()